# ДЗ 11


1. Взять предобученную трансформерную архитектуру и решить задачу перевода (для того же корпуса что вы выбрали из предыдущего дз)

In [1]:
!pip install transformers
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.1 MB/s 
     |████████████████████████████████| 101 kB 2.8 MB/s 
     |████████████████████████████████| 6.6 MB 39.2 MB/s 
     |████████████████████████████████| 596 kB 40.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.8 MB/s 


In [2]:
import io
import re

### 1. Взять предобученную трансформерную архитектуру и решить задачу перевода (для того же корпуса что вы выбрали из предыдущего дз)

In [3]:
!wget http://www.manythings.org/anki/rus-eng.zip

--2022-08-14 14:01:58--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14819554 (14M) [application/zip]
Saving to: ‘rus-eng.zip’

rus-eng.zip         100%[===================>]  14.13M  13.3MB/s    in 1.1s    

2022-08-14 14:02:00 (13.3 MB/s) - ‘rus-eng.zip’ saved [14819554/14819554]



In [4]:
!mkdir rus-eng
!unzip rus-eng.zip -d rus-eng/

Archive:  rus-eng.zip
  inflating: rus-eng/rus.txt         
  inflating: rus-eng/_about.txt      


In [5]:
path_to_file = "rus-eng/rus.txt"

In [6]:
def preprocess_sentence(w):
  w = w.lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [7]:
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [8]:
en, ru = create_dataset(path_to_file, None)
print(en[20])
print(ru[20])

<start> duck ! <end>
<start> пригнись ! <end>


In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en", do_lower_case=True)
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [10]:
def translate(sentence):

  inputs = tokenizer.encode(sentence, return_tensors="pt")
  outputs = model.generate(inputs, max_length=40, num_beams=4, early_stopping=True)

  result = tokenizer.decode(outputs[0])

  return result

In [11]:
# хаотично беру предложения из корпуса с прошлого (10) домашнего задания.

for i in [5, 20, 50, 100, 256, 523, 526, 548, 4536, 62458, 46358, 1596]:
    print(ru[i], translate(ru[i]))

<start> хай . <end> <pad> <start> hey. <end>
<start> пригнись ! <end> <pad> <start> get down! <end>
<start> продолжайте . <end> <pad> <start> continue. <end>
<start> съешьте это . <end> <pad> <start> eat this. <end>
<start> сложи его . <end> <pad> <start> folding it. <end>
<start> не ездите . <end> <pad> <start> do not go. <end>
<start> едем дальше . <end> <pad> <start> moving on. <end>
<start> убирайся ! <end> <pad> #start> get out! #end>
<start> я пакую вещи . <end> <pad> <start> I pack things. <end>
<start> том слишком много говорит . <end> <pad> <start> volume speaks too much. <end>
<start> спасибо за ужин . <end> <pad> #start> Thank you for dinner. <end>
<start> так оно и есть . <end> <pad> <start> so it is. <end>


На мой взгляд предобученная трансформерная архитектура справилась с переводом очень неплохо, кроме одной строки, думаю дело в имени
